In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
import register_ as reg
import algo_cooling as algo
import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from quimb.experimental import tnvmc
from tqdm import tqdm

In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False)
# opt = "auto-hq"

In [3]:
B = 10_00
seed = 42
L = 8
bond_dim = 4
chi = 4

In [4]:

p_ = qtn.MPS_rand_state(L=L, bond_dim=bond_dim, seed=3, dtype="float64") 
p_.normalize()
p_.apply_to_arrays(to_backend)
p_.left_canonize()


MatrixProductState(tensors=8, indices=15, L=8, max_bond=4)

In [5]:
p = qtn.MPS_rand_state(L=L, bond_dim=chi, seed=6, dtype="float64")
p.apply_to_arrays(to_backend)

p.normalize()
print(f"Site tags: '{p.site_tag_id}', site inds: '{p.site_ind_id}'")

Site tags: 'I{}', site inds: 'k{}'


In [6]:
(p.H & p).contract(all, optimize=opt), (p_.H & p_).contract(all, optimize=opt)

(tensor(1.0000, dtype=torch.float64), tensor(1.0000, dtype=torch.float64))

In [7]:
algo.fidel_mps(p, p_)

0.0008023878045451766

In [8]:
sampler = p.sample(B, seed=seed)

In [9]:
%%time
samples_l = []
ampl_l = []
w_l = []
for i in tqdm(sampler):
    basis, w = i
    w_l.append(w)
    p_0 = qtn.MPS_computational_state(basis)
    p_0.apply_to_arrays(to_backend)
    samples_l.append(p_0)
    
    amp = (p.H | p_0).contract(all, optimize=opt)
    ampl_l.append(amp)
    # print(amp, amp**2, w)

1000it [00:04, 229.07it/s]

CPU times: user 17.9 s, sys: 26.5 ms, total: 17.9 s
Wall time: 4.37 s


In [10]:
def renormalize_(p_, samples_l, w_l):
    norm_l = []
    for count, state in enumerate(samples_l): 
        amp = (state.H | p_).contract(all, optimize=opt)
        norm_l.append( amp**2 / w_l[count] )
    return sum(norm_l) / len(norm_l)


def normalize_state(p_):
    norm_ = renormalize_(p_, samples_l, w_l)
    return p_ / norm_ ** 0.5
    # return p_ / (p_.H @ p_) ** 0.5

def loss(p_, samples_l, ampl_l, opt, w_l):

    # norm_l = []
    # for count, state in enumerate(samples_l): 
    #     amp = (state.H | p_).contract(all, optimize=opt)
    #     norm_l.append( amp**2 / w_l[count] )
    # norm_ = sum(norm_l) / len(norm_l)
    # p_ = p_ / norm_ ** 0.5
    # print(p_.norm(), norm_)
    
    cost_l = []
    f = 0
    for count, state in enumerate(samples_l):
        
        val = (state.H | p_).contract(all, optimize=opt)
        cost_l.append( val/ampl_l[count] )
        f += val 

    f = sum(cost_l) / len(samples_l)
    
    return - abs( f ) ** 2

In [11]:
%%time
norm_ = renormalize_(p_, samples_l, w_l)
norm_

CPU times: user 983 ms, sys: 1.1 ms, total: 984 ms
Wall time: 982 ms


tensor(0.7234, dtype=torch.float64)

In [12]:
%%time
dis = loss(p_, samples_l, ampl_l, opt, w_l)

CPU times: user 988 ms, sys: 208 µs, total: 988 ms
Wall time: 986 ms


In [13]:
complex(dis)

(-0.0003060017811297163+0j)

In [14]:

tnopt = qtn.TNOptimizer(
    p_,                        # the tensor network we want to optimize
    loss_fn=loss,
    norm_fn=normalize_state,
    loss_constants = { "samples_l":samples_l, "w_l":w_l},# the function we want to minimize
    loss_kwargs = {'opt': opt, "ampl_l":ampl_l},
    autodiff_backend = "torch",  # 'autograd',   # use 'autograd' for non-compiled optimization
    optimizer='L-BFGS-B', #'adam', 'L-BFGS-B'    # the optimization algorithm
    progbar=True, jit_fn=False, device="cpu", 
)


/home/reza.haghshenas@quantinuum.com/.local/lib/python3.10/site-packages/quimb/tensor/optimize.py:632: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x).to(self.device)


In [15]:
p_ = tnopt.optimize(n=100, ftol=1.e-12, maxfun= 10e+10, gtol= 1e-14, eps=1.e-10, maxls=500, iprint = 0, disp=False)
p_.apply_to_arrays(to_backend)


-0.997969119208 [best: -0.997969119208] : : 106it [05:54,  3.35s/it]                                                                   


In [16]:
algo.fidel_mps(p, p_)

0.9738639031012175

In [17]:
dis = loss(p_, samples_l, ampl_l, opt, w_l)
dis

tensor(-0.9980, dtype=torch.float64)

In [18]:
p_.norm()

tensor(1.0123, dtype=torch.float64)